In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_dir = "/content/drive/My Drive/Polygence/Audrey/"

In [0]:
% matplotlib inline
import matplotlib.pyplot as plt
import json
import nltk
import numpy as np
import pandas as pd
import glob
from collections import Counter

In [0]:
candidates = open(root_dir + "data/candidate_names.txt", "r").read().splitlines() + ["Michael Bloomberg"]
sources = open(root_dir + "data/source_names.txt", "r").read().splitlines()

### Load combined articles

In [0]:
df = pd.read_csv(root_dir + "data/coref_resolved_all.csv")
df.head()

,Unnamed: 0,source,author,title,url,publishedAt,content,coref_resolved
0,0,abc-news,John Verhovek,5 takeaways from the 2nd night of the Democrat...,https://abcnews.go.com/Politics/takeaways-2nd-...,2019-08-01T03:31:25Z,\nEven before all the candidates took the stag...,\nEven before all the candidates took the stag...
1,1,abc-news,ABC NEWS,Fact-checking Democratic candidates on the iss...,https://abcnews.go.com/Politics/democratic-deb...,2019-08-01T01:29:38Z,\nHere's ABC News' fact check of the second of...,\nHere's ABC News' fact check of the second of...
2,2,abc-news,Kendall Karson and Elizabeth Thomas,What to expect from 2nd night of Democratic de...,https://abcnews.go.com/Politics/democratic-deb...,2019-07-31T20:01:36Z,\nAfter Sens. Bernie Sanders and Elizabeth War...,\nAfter Sens. Bernie Sanders and Elizabeth War...
3,3,abc-news,Soo Rin Kim,2020 candidates spend millions on online ads a...,https://abcnews.go.com/Politics/love-hate-rela...,2019-07-27T10:22:00Z,\nDemocrats vying for presidential election in...,\nDemocrats vying for presidential election in...
4,4,abc-news,Rick Klein and MaryAlice Parks,The Note: Blue wave continues ride through Ken...,https://abcnews.go.com/Politics/note-blue-wave...,2019-11-06T11:05:10Z,\nThe TAKE with Rick Klein\n \nIt might just b...,\nThe TAKE with Rick Klein\n \nIt might just b...


In [0]:
all_articles = " ".join(df["coref_resolved"])

In [0]:
candidate_counts = {}
for c in candidates:
  last = c.split(' ')[-1]
  count = all_articles.count(last)
  if count > 300:
    candidate_counts[last] = count
print(candidate_counts)
  

{'Bennet': 2095, 'Biden': 59515, 'Blasio': 4176, 'Booker': 9305, 'Bullock': 2085, 'Buttigieg': 20375, 'Castro': 6031, 'Delaney': 1972, 'Gabbard': 7942, 'Gillibrand': 1949, 'Harris': 16639, 'Inslee': 1061, 'Klobuchar': 7647, 'Moulton': 496, "O'Rourke": 3185, 'Ryan': 2621, 'Sanders': 38827, 'Steyer': 4732, 'Warren': 44632, 'Williamson': 2393, 'Yang': 7978, 'Bloomberg': 4911}


In [0]:
len(candidate_counts)

22

## Dependency parse (new code)

In [0]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [0]:
def get_verbs(doc, word):
  verbs = []

  # extract verbs
  for token in doc:
    if token.pos_ != "VERB":
      continue
    for child in token.children:
      if child.text == word:
        verbs.append(token.text.lower())
  counts = Counter(verbs)
  return "\n".join(["%s %d" % (verb, count) for verb, count in counts.most_common()])

In [0]:
def get_adjectives(doc, word):
  adjectives = []
  # extract adjectives
  for token in doc:
    if token.dep_ == "amod":
      if token.head.text == word:
        adjectives.append(token.text.lower())
  counts = Counter(adjectives)
  return "\n".join(["%s %d" % (adjective, count) for adjective, count in counts.most_common()])

In [0]:
parsed = df["coref_resolved"].apply(nlp)

In [0]:
len(parsed)

12600

In [0]:
parsed[:10]

0    (\n, Even, before, all, the, candidates, took,...
1    (\n, Here, 's, ABC, News, ', fact, check, of, ...
2    (\n, After, Sens, ., Bernie, Sanders, and, Eli...
3    (\n, Democrats, vying, for, presidential, elec...
4    (\n, The, TAKE, with, Rick, Klein, \n \n, It, ...
5    (\n, The, TAKE, with, Rick, Klein, \n \n, It, ...
6    (\n, The, TAKE, with, Rick, Klein, \n \n, Iran...
7    (\n, Democratic, contenders, seeking, to, repl...
8    (\n, The, TAKE, with, Rick, Klein, \n \n, Rick...
9    (DES, MOINES, ,, Iowa, --, \n, Increasingly, a...
Name: coref_resolved, dtype: object

In [0]:
for a in candidate_counts:
  print(a)
  df[a + "_verbs"] = parsed.apply(get_verbs, args=(a,))
  print(df[a + "_verbs"].head())
  df[a + "_adjectives"] = parsed.apply(get_adjectives, args=(a,))
  print(df[a + "_adjectives"].head())

Bennet
0                                        had 1\nsaid 1
1                                                     
2    said 5\nchimed 1\nresponded 1\ncontinued 1\ncr...
3                                             burned 1
4                                                     
Name: Bennet_verbs, dtype: object
0    
1    
2    
3    
4    
Name: Bennet_adjectives, dtype: object
Biden
0    said 3\nsparred 1\ncriticized 1\nhit 1\nlearne...
1    said 4\nwas 2\nwrote 2\nresponded 2\nvoted 2\n...
2    said 11\nresponded 4\nwas 3\nis 2\nasked 2\npu...
3                                become 1\nincluding 1
4                                                     
Name: Biden_verbs, dtype: object
0         
1         
2    old 1
3         
4         
Name: Biden_adjectives, dtype: object
Blasio
0                                  warned 1\nbelieve 1
1                                                     
2    said 3\noppose 1\nmake 1\npressed 1\nasked 1\n...
3                                   

In [0]:
df.to_csv(root_dir + "data/all_articles_dependencies.csv", index=False)

### Remove punctuation and stopwords

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
import re
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [0]:
punct_chars = list((set(string.punctuation) | {'’', '‘', '–', '—', '~', '|', '“', '”', '…', "'", "`", '_'}) - set(['#']))
punct_chars.sort()
punctuation = ''.join(punct_chars)
replace = re.compile('[%s]' % re.escape(punctuation))


def clean_text(text):
    # lower case
    text = text.lower()
    # eliminate urls
    text = re.sub(r'http\S*|\S*\.com\S*|\S*www\S*', ' ', text)
    # substitute all other punctuation with whitespace
    text = replace.sub(' ', text)
    # replace all whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    # strip off spaces on either end
    text = text.strip()
    words = text.split()
    words = [w for w in words if w not in stop]
    return words

In [0]:
all_articles_cleaned = clean_text(all_articles)

In [0]:
all_articles_cleaned[:10] # this should be a list of words

['westerville',
 'ohio',
 'squad',
 'bernie',
 'back',
 'three',
 'four',
 'famous',
 'freshman',
 'congresswomen']

## Check words surrounding each candidate

In [0]:
names = [n.lower() if "Rourke" not in n else "rourke" for n in candidate_counts.keys()]
print(names)

['biden', 'blasio', 'booker', 'bullock', 'buttigieg', 'castro', 'delaney', 'gabbard', 'gillibrand', 'harris', 'klobuchar', 'rourke', 'ryan', 'sanders', 'warren', 'williamson', 'yang']


In [0]:
surround = {}
for a in names:
  surround[a] = []
  for idx, word in enumerate(all_articles_cleaned):
    if word == a:
      surround[a].append(all_articles_cleaned[idx-1])
      surround[a].append(all_articles_cleaned[idx+1])
      surround[a].append(all_articles_cleaned[idx-2])
      surround[a].append(all_articles_cleaned[idx+2])


In [0]:
print(surround)

{'biden': ['joe', 'said', 'president', 'smile', 'harris', 'targets', 'trump', 'constant', 'criticized', 'criminal', 'sharply', 'justice', 'hit', 'plan', 'inslee', 'combat', 'said', 'learned', 'castro', 'lessons', 'debate', 'harris', 'cooling', 'wednesday', 'said', 'criticized', 'plan', 'harris', 'president', 'simply', 'vice', 'inaccurate', 'following', 'critique', 'said', 'plan', 'stage', 'harris', 'fact', 'lobbing', 'aim', 'women', 'took', 'rights', 'implored', 'approach', 'inslee', 'issue', 'ganged', 'continually', 'candidates', 'defended', 'president', 'said', 'running', 'one', 'went', 'issue', 'also', 'immigration', 'attacks', 'dissipate', 'expect', 'candidates', 'critiquing', 'record', 'sharply', 'mr', 'said', 'pointed', 'booker', 'record', 'attacks', 'booker', 'cogent', 'able', 'harris', 'offered', 'neither', 'answer', 'criticism', 'allowed', 'night', 'distinguish', 'pleading', 'heed', 'said', 'warnings', 'joe', 'sen', 'president', 'kamala', 'check', 'sen', 'fact', 'harris', 'way

In [0]:
for k, v in surround.items():
  surround[k] = Counter(v).most_common()

In [0]:
for k, v in surround.items():
  print(k)
  print(v[:20])
  print('--------')

biden
[('joe', 1133), ('mr', 636), ('president', 635), ('said', 446), ('harris', 218), ('debate', 169), ('campaign', 166), ('also', 116), ('booker', 111), ('sanders', 107), ('trump', 105), ('joseph', 101), ('aposs', 100), ('jr', 95), ('r', 94), ('biden', 82), ('vice', 81), ('sen', 79), ('obama', 79), ('would', 76)]
--------
blasio
[('de', 1005), ('bill', 343), ('said', 125), ('mr', 123), ('mayor', 50), ('new', 46), ('also', 31), ('aposs', 26), ('told', 24), ('york', 20), ('asked', 19), ('tweeted', 19), ('democratic', 17), ('city', 16), ('biden', 15), ('one', 15), ('responded', 15), ('campaign', 13), ('hannity', 13), ('debate', 12)]
--------
booker
[('cory', 572), ('sen', 266), ('mr', 159), ('said', 157), ('senator', 129), ('new', 127), ('biden', 111), ('jersey', 107), ('harris', 90), ('n', 66), ('j', 66), ('nj', 53), ('mayor', 39), ('also', 34), ('former', 32), ('campaign', 30), ('aposs', 26), ('trump', 23), ('back', 23), ('says', 22)]
--------
bullock
[('steve', 198), ('gov', 138), ('

## Look at verbs and adjectives associated with each candidate

### Run dependency parsing (no need to do it again; skip to the next section to load it)

In [0]:
all_articles[:100]


'\nEven before all the candidates took the stage in Detroit on Wednesday for the second night of the l'

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import spacy

nlp = spacy.load("en_core_web_sm")

def get_dependents(word, docs):
  adjectives = []
  verbs = []

  for doc in docs:
    # extract adjectives
    for token in doc:
      if token.dep_ == "amod":
        if token.head.text == word:
          adjectives.append(token.text.lower())

    # extract verbs
    for token in doc:
      if token.pos_ != "VERB":
        continue
      for child in token.children:
        if child.text == word:
          verbs.append(token.text.lower())
  print(adjectives)
  print(verbs)
  return adjectives, verbs
        

Since the document is too big, we can't run dependency parsing on it all together. We need to do it in batches.

In [0]:
# Step 1: split document into sentences
sentences = nltk.sent_tokenize(all_articles)
len(sentences)

347263

In [0]:
# Step 2: parse sentences in batches
batch_size=500
i = 0
docs = []
while i < len(sentences):
  if i % 10000 == 0:
    print(i)
  # Rejoin batch of sentences into a document
  curr_sentences = " ".join(sentences[i:i+batch_size])
  docs.append(nlp(curr_sentences))
  i+=batch_size

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000


In [0]:
len(docs)

695

### Save dependency parsed documents

In [0]:

import pickle

# Step 3: save parsed sentences in batches
batch_size=200
i = 0
count = 0
while i < len(docs):
  print(count)
  with open(root_dir + 'data/depparse_' + str(count) + '.pickle', 'wb') as handle:
    pickle.dump(docs[i:i+batch_size], handle, protocol=pickle.HIGHEST_PROTOCOL)
  i+=batch_size
  count +=1


0
1
2
3


### Load dependency parsed documents

In [0]:
import pickle
docs = []
for f in glob.glob(root_dir + 'data/depparse/*.pickle'):
  with open(f, "rb") as handle:
    docs.extend(pickle.load(handle))

In [0]:
len(docs)

600

### Obtain verbs and adjectives

In [0]:
verbs = {}
adjectives = {}
for a in candidate_counts:
  print(a)
  adjectives[a], verbs[a] = get_dependents(a, docs)

Bennet
['young', 'young', 'young', 'young', 'young', 'hopeful', 'known', 'said', 'said', 'echoed', 'refrain', 'true', 'true', 'clear']
['hit', 'wrote', 'arrived', 'mentioned', 'is', 'threatened', 'thought', 'needs', 'break', 'is', 'says', 'follow', 'blasted', 'said', 'advocated', 'brought', 'walked', 'told', 'were', 'agreed', 'threaten', 'snarked', 'qualify', 'told', 'run', 'rallying', 'called', 'handle', 'handle', 'told', 'said', 'know', 'be', 'have', 'does', 'appreciates', 'meet', 'rejected', 'said', 'premised', 'said', 'boost', 'unveiled', 'invest', 'said', 'polling', 'said', 'plans', 'is', 'get', 'is', 'proved', 'be', 'has', 'is', 'end', 'has', 'consider', 'has', 'was', 'announced', 'had', 'said', 'said', 'said', 'put', 'asked', 'smoked', 'blocked', 'kept', 'fought', 'chimed', 'responded', 'continued', 'criticize', 'said', 'said', 'burned', 'been', 'said', 'had', 'pay', 'were', 'include', 'derided', 'said', 'came', 'said', 'said', 'including', 'including', 'said', 'is', 'said', 'is

In [0]:
# Save adjectives
for k, v in adjectives.items():
  adjectives[k] = Counter(v)
with open(root_dir + "data/adjectives.json", "w") as f:
  f.write(json.dumps(adjectives))

In [0]:
# Save verbs
for k, v in verbs.items():
  verbs[k] = Counter(v)
with open(root_dir + "data/verbs.json", "w") as f:
  f.write(json.dumps(verbs))

## Identify most distinctive words for each candidate

In [0]:
from collections import defaultdict
import math
import operator

In [0]:
def log_odds(counts1, counts2, prior, zscore = True):
  """Calculates and prints out the most distintive words between two groups.
  
  Arguments:
    counts1: dictionary of word counts for group 1
    counts2: dictionary of word counts for group 2
    prior: dictionary of word counts for all the data
    zscore: whether to normalize the final score (good to keep it as "True")
  """

  sigmasquared = defaultdict(float)
  sigma = defaultdict(float)
  delta = defaultdict(float)

  n1 = sum(counts1.values())
  n2 = sum(counts2.values())

  # since we use the sum of counts from the two groups as a prior, this is equivalent to a simple log odds ratio
  nprior = sum(prior.values())
  for word in prior.keys():
    if prior[word] == 0:
      delta[word] = 0
      continue
    l1 = float(counts1[word] + prior[word]) / (( n1 + nprior ) - (counts1[word] + prior[word]))
    l2 = float(counts2[word] + prior[word]) / (( n2 + nprior ) - (counts2[word] + prior[word]))
    sigmasquared[word] = 1/(float(counts1[word]) + float(prior[word])) + 1/(float(counts2[word]) + float(prior[word]))
    sigma[word] = math.sqrt(sigmasquared[word])
    delta[word] = (math.log(l1) - math.log(l2))
    if zscore:
      delta[word] /= sigma[word]
  
  sorted_scores = sorted(delta.items(), key=operator.itemgetter(1))
  
    
  print("Most asssociated words:")
  for k, v in reversed(sorted_scores[-20:]):
    if v >= 2:
      print(k, v)
  
  print()
  print("Least associated words:")
  for k, v in sorted_scores[:10]:
    if v <= -2:
      print(k, v)
  
  return delta

TODO for Audrey: 

1. Combine verbs and adjectives for each candidate into a single list
2. For each candidate, create a dictionary of word counts for that candidate (use Counter) and create a dictionary of word counts combining words for ALL OTHER candidates. For example, create a dictionary of word counts for Warren and create a dictionary of word counts that includes words for all candidates except for Warren.
3. Create a dictionary of overall word counts (combining words for all candidates)
4. For each candidate, run the `log_odds()` function above: `counts1` will be the dictionary of counts for that candidate and `counts2` will be the dictionary of counts for all other candidates. `prior` will be the dictionary of overall word counts. 
5. Look at the words that come out and see if you notice anything interesting.

In [0]:
combined_words = {}
for a in candidate_counts:
  adjectives, verbs = get_dependents(a,docs)
  combined_words[a] = adjectives+verbs
  print(a)
  print(combined_words[a])

['young', 'young', 'young', 'young', 'young', 'hopeful', 'known', 'said', 'said', 'echoed', 'refrain', 'true', 'true', 'clear']
['hit', 'wrote', 'arrived', 'mentioned', 'is', 'threatened', 'thought', 'needs', 'break', 'is', 'says', 'follow', 'blasted', 'said', 'advocated', 'brought', 'walked', 'told', 'were', 'agreed', 'threaten', 'snarked', 'qualify', 'told', 'run', 'rallying', 'called', 'handle', 'handle', 'told', 'said', 'know', 'be', 'have', 'does', 'appreciates', 'meet', 'rejected', 'said', 'premised', 'said', 'boost', 'unveiled', 'invest', 'said', 'polling', 'said', 'plans', 'is', 'get', 'is', 'proved', 'be', 'has', 'is', 'end', 'has', 'consider', 'has', 'was', 'announced', 'had', 'said', 'said', 'said', 'put', 'asked', 'smoked', 'blocked', 'kept', 'fought', 'chimed', 'responded', 'continued', 'criticize', 'said', 'said', 'burned', 'been', 'said', 'had', 'pay', 'were', 'include', 'derided', 'said', 'came', 'said', 'said', 'including', 'including', 'said', 'is', 'said', 'is', 'sai

In [0]:
# dictionary of word counts for each candidate
candidate_word_counts = {} 
for key in candidate_counts:
    candidate_word_counts[key] = Counter(combined_words[key])

print(candidate_word_counts['Biden'])
print(candidate_word_counts['Sanders'])

Counter({'said': 1682, 'was': 693, 'is': 680, 'investigate': 533, 'told': 315, 'has': 302, 'had': 256, 'endorses': 190, 'says': 188, 'running': 183, 'be': 162, 'did': 157, 'called': 154, 'made': 154, 'calls': 142, 'have': 141, 'uses': 140, 'accused': 133, 'took': 130, 'asked': 130, 'leads': 123, 'are': 114, 'including': 111, 'been': 109, 'added': 108, 'gain': 104, 'criticized': 96, 'responded': 95, 'leading': 92, 'served': 87, 'raised': 81, 'defended': 81, 'attacked': 81, 'wrote': 77, 'needs': 74, 'came': 71, 'spent': 69, 'led': 69, 'extends': 69, 'showed': 68, 'got': 66, 'left': 65, 'claimed': 64, 'spoke': 63, 'were': 62, 'investigating': 61, 'thank': 61, 'hopeful': 60, 'going': 60, 'support': 59, 'found': 59, 'shows': 59, 'take': 57, 'joined': 57, 'continued': 57, 'used': 56, 'holds': 56, 'lost': 55, 'announced': 55, 'held': 55, 'attacking': 55, 'went': 55, 'remains': 54, 'put': 54, 'helped': 54, 'tried': 53, 'continues': 53, 'appeared': 53, 'denied': 53, 'done': 52, 'faced': 52, 'ga

In [0]:
# dictionary of word counts except for each candidate
candidate_not_word_counts = {}
for key in candidate_counts:
  words = []
  for name in candidate_counts:
    if key != name:
      words.extend(combined_words[name])
  candidate_not_word_counts[key] = Counter(words)
print(candidate_not_word_counts['Biden'])
print(candidate_not_word_counts['Blasio'])
print(candidate_not_word_counts['Booker'])

Counter({'said': 7488, 'is': 2368, 'was': 1721, 'told': 1194, 'has': 893, 'says': 765, 'called': 677, 'had': 643, 'asked': 606, 'added': 588, 'made': 474, 'be': 450, 'have': 415, 'took': 406, 'hopeful': 396, 'wrote': 385, 'responded': 385, 'tweeted': 377, 'are': 359, 'including': 352, 'endorsed': 336, 'criticized': 335, 'been': 320, 'accused': 302, 'announced': 293, 'wants': 285, 'raised': 278, 'did': 272, 'released': 269, 'becomes': 268, 'put': 264, 'spent': 255, 'continued': 244, 'came': 229, 'went': 217, 'spoke': 214, 'argued': 211, 'got': 211, 'defended': 207, 'beat': 205, 'supporting': 198, 'were': 197, 'proposed': 197, 'supports': 192, 'thank': 190, 'support': 175, 'gain': 173, 'believes': 171, 'hit': 164, 'going': 160, 'calls': 160, 'want': 156, 'suggested': 155, 'needs': 153, 'claimed': 152, 'do': 151, 'used': 149, 'met': 147, 'reported': 147, 'found': 145, 'attacked': 145, 'joined': 145, 'replied': 144, 'received': 141, 'running': 141, 'faced': 140, 'calling': 138, 'challenged

In [0]:
# total word counts
all_words = []
for key in candidate_counts:
  all_words.extend(combined_words[key])

all_words_count = Counter(all_words)
print(all_words_count)

Counter({'said': 9170, 'is': 3048, 'was': 2414, 'told': 1509, 'has': 1195, 'says': 953, 'had': 899, 'called': 831, 'asked': 736, 'added': 696, 'made': 628, 'be': 612, 'have': 556, 'took': 536, 'investigate': 533, 'responded': 480, 'are': 473, 'including': 463, 'wrote': 462, 'hopeful': 456, 'accused': 435, 'criticized': 431, 'been': 429, 'did': 429, 'tweeted': 416, 'endorsed': 374, 'raised': 359, 'announced': 348, 'spent': 324, 'running': 324, 'wants': 322, 'put': 318, 'released': 306, 'calls': 302, 'continued': 301, 'came': 300, 'defended': 288, 'spoke': 277, 'got': 277, 'gain': 277, 'becomes': 276, 'went': 272, 'were': 259, 'argued': 251, 'thank': 251, 'beat': 246, 'support': 234, 'supports': 232, 'leads': 229, 'needs': 227, 'attacked': 226, 'going': 220, 'proposed': 217, 'claimed': 216, 'supporting': 212, 'hit': 209, 'leading': 209, 'used': 205, 'found': 204, 'suggested': 202, 'joined': 202, 'endorses': 200, 'do': 197, 'believes': 195, 'want': 192, 'faced': 192, 'take': 185, 'appeare

In [0]:
deltas = {}
for key in candidate_counts:
  print(key)
  deltas[key] = log_odds(candidate_word_counts[key], candidate_not_word_counts[key], all_words_count)
  print("-------")
  print()

Bennet
Most asssociated words:
thank 3.881805439448973

Least associated words:
-------

Biden
Most asssociated words:
investigate 19.68934070392536
endorses 11.28711493867675
uses 8.95732557490627
extends 6.904432632870169
running 6.7421672612865455
investigating 6.623433257234265
leads 5.1794324153803055
served 5.065004106512152
younger 4.799524179657038
calls 4.606109419945488
elder 4.565115576836132
warn 4.238401593545444
shows 4.216656179161292
denied 4.114115503824828
step 4.047350917243318
corrected 3.897817376181253
national 3.596103378855545
attacking 3.565418518208712
engaged 3.5054655652378894
probe 3.4947427077200066

Least associated words:
said -7.618061224033887
becomes -4.182776743731069
tweeted -3.601578160711268
qualifies -3.2733282023252133
endorsed -3.2354440241558295
praises -3.0943820936918196
supporting -3.035723025677272
hopeful -2.824994477832372
added -2.766488660626069
wants -2.7207936572273237
-------

Blasio
Most asssociated words:
begin 2.5330979441785204
